In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
###############################################################################
########################## UTILITY FUNCTIONS ##################################
###############################################################################

def swap_rows(M,row1,row2):
    '''
    Swap the rows of 2D matrix M in place.
    '''
    M[[row1,row2]] = M[[row2,row1]]
    
def find_pivot_row(X,n=0):
    '''
    Get index of row with the largest (by magnitude) element in column n.
    '''
    return np.argsort(-np.abs(X[n:,n]))[0] + n


def perm_mat(N,row1,row2):
    '''
    Return the permutation matrix that Swaps row1 and row2 for an NxN matrix.
    '''
    P = np.eye(N)
    swap_rows(P,row1,row2)
    return P

###############################################################################
########################## MAIN STEPS #########################################
###############################################################################


def permutation_step(DU,L=None,P=None,n=0):
    '''
    Permute X to place highest magnitude pivot of X in current row (n) by swapping,
    and permute L to keep row correspondence and lower triangular form. Record 
    permutation in P.
    
    PA = LDU
    this step permutation: Pn
    (Pn P)A = (Pn L Pn)(Pn DU)
    '''
    if P is None:
        P = np.eye(len(DU))
    if L is None:
        L = np.eye(len(DU))
        
    pivot = find_pivot_row(X=DU,n=n)
    if pivot != n:
        # Find permutation for this step
        Pn = perm_mat(len(DU),pivot,n)
        # Update total permutation matrix -> Pn(PA)
        P = np.matmul(Pn,P)
        DU = np.matmul(Pn,DU)
        L = np.matmul(Pn,np.matmul(L,Pn))
    
    return P,L,DU


def elimination_step(DU,L=None,n=0):
    '''
    Perform Gaussian elimination on rows below pivot row (n) such that
    the only nonzero element of the pivot column is the pivot.
    '''
    if L is None:
        L = np.eye(len(DU))

    for nn in range(n+1,len(DU)):
        if DU[n,n] != 0:
            factor = DU[nn,n]/DU[n,n]
            DU[nn] -= factor*DU[n]  # eliminate element in pivot column by subtracting 
                                            # scaled pivot row from each lower row
            L[nn,n] = factor        # Store (inverse) elementary operations in L

    return L,DU
   
def factor_out_D(DU):
    '''
    DU should be an upper triangular matrix. This function will 
    factor out the elements on the diagonal so they are set to 1.
    The diagonal elements are returned in D, the remaining upper
    triangular matrix (with 1s on the diag) in U.
    '''    
    diag = np.diag(DU)  # extract the diagonal of DU
    D = np.diag(diag)  # make into a diagonal matrix
    U = DU / diag[:,None]  # factor out D from DU
    
    return D,U

###############################################################################
################## TESTING FUNCTIONS ##########################################
###############################################################################


def sp_ldu(A):
    from scipy.linalg import lu
    inv_p,l,du = lu(A)
    p = np.linalg.inv(inv_p)
    d,u = factor_out_D(du)
    return p,l,d,u

def test_decomp(A,P,L,D,U):
    p,l,d,u = sp_ldu(A)
    assert (p == P).all()
    assert (l == L).all()
    assert (d == D).all()
    assert (u == U).all()
    print("Success!")
    
def print_LDU(A,P,L,D,U):
    print('A')
    print(A)
    print()
    print('P')
    print(P)
    print('DU')
    print(np.matmul(D,U))
    print('L')
    print(L)
    print('D')
    print(D)
    print('U')
    print(U)
    print()
    print('PA')
    print(np.matmul(P,A))
    print('LDU')
    print(np.matmul(L,np.matmul(D,U)))

In [50]:
## Matrices from hw1 problem 2

A = np.array(
    [[4, 7, 0],
     [3, 2, 1],
     [2, 2, -6]
    ],
    dtype='double'
)

B = np.array(
    [[1,0,0,0,1],
     [0,0,1,0,0],
     [0,1,0,1,0],
     [0,1,0,0,0],
     [1,0,0,0,0],
    ],
    dtype='double'
)


C = np.array(
    [[2, 2, 5],
     [3, 2, 5],
     [1, 1, 5]
    ],
    dtype='double'
)

In [51]:
def LDU(A):
    # P and L begin as identity matrices
    P = np.eye(len(A)) # assuming square matrix
    L = np.eye(len(A))
    DU = A.copy() # this matrix will have L factored out until it is DU

    for n in range(len(A-1)):
        P,L,DU = permutation_step(DU=DU,L=L,P=P,n=n)
        L,DU   = elimination_step(DU=DU,L=L,n=n)
    D,U = factor_out_D(DU)
    
    return P, L, D, U

X = [A,B,C]
for x in X:
    P,L,D,U = LDU(x)
    test_decomp(A=x,P=P,L=L,D=D,U=U) # Check against built-in method



Success!
Success!
Success!


In [58]:
[print("{}\n{}\n\n".format(x,y)) for x,y in zip(LDU(C),sp_ldu(C))];

[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]
[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]


[[1.         0.         0.        ]
 [0.66666667 1.         0.        ]
 [0.33333333 0.5        1.        ]]
[[1.         0.         0.        ]
 [0.66666667 1.         0.        ]
 [0.33333333 0.5        1.        ]]


[[3.         0.         0.        ]
 [0.         0.66666667 0.        ]
 [0.         0.         2.5       ]]
[[3.         0.         0.        ]
 [0.         0.66666667 0.        ]
 [0.         0.         2.5       ]]


[[1.         0.66666667 1.66666667]
 [0.         1.         2.5       ]
 [0.         0.         1.        ]]
[[1.         0.66666667 1.66666667]
 [0.         1.         2.5       ]
 [0.         0.         1.        ]]




In [5]:
# HW 1 #2: Compute SVDs
for n,x in enumerate(X):
    print("\n\n----------------------\n#{}:".format(n+1))
    print(x)
    u,s,v = np.linalg.svd(x,full_matrices=True)
    print("\n\nU:\n")
    print(np.round(u,3))
    print("\nS:")
    print(np.round(s,3))
    print("\nV:")
    print(np.round(v,3))



----------------------
#1:
[[ 4.  7.  0.]
 [ 3.  2.  1.]
 [ 2.  2. -6.]]


U:

[[ 0.831  0.364 -0.421]
 [ 0.321  0.304  0.897]
 [ 0.454 -0.88   0.136]]

S:
[9.331 5.79  1.555]

V:
[[ 0.557  0.79  -0.258]
 [ 0.105  0.241  0.965]
 [ 0.824 -0.564  0.051]]


----------------------
#2:
[[1. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 1. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]]


U:

[[-0.851  0.     0.    -0.526  0.   ]
 [ 0.     0.    -1.     0.     0.   ]
 [ 0.    -0.851  0.     0.    -0.526]
 [ 0.    -0.526  0.     0.     0.851]
 [-0.526  0.     0.     0.851  0.   ]]

S:
[1.618 1.618 1.    0.618 0.618]

V:
[[-0.851  0.     0.     0.    -0.526]
 [-0.    -0.851 -0.    -0.526  0.   ]
 [-0.    -0.    -1.    -0.     0.   ]
 [ 0.526  0.     0.     0.    -0.851]
 [ 0.     0.526  0.    -0.851  0.   ]]


----------------------
#3:
[[2. 2. 5.]
 [3. 2. 5.]
 [1. 1. 5.]]


U:

[[-0.586 -0.044 -0.809]
 [-0.623 -0.614  0.485]
 [-0.518  0.788  0.332]]

S:
[9.791 1.416 0.361]

V:
[[-0.364 -0.3   -0.882]


In [42]:
# HW #3 -- solve Ax=b systems

def unique_soln(A,dec=4):
    d = round(np.linalg.det(A),dec)
    if d == 0:
        print("det(A)==0 (up to {} dec) -> A is singular, no unique solution exists.".format(dec))
        return False
    else:
        print("det(A)!=0 ({})-> A is non-singular, a unique solution exists.".format(d))
        return True
              
A3a = np.array([
    [2,1,3],
    [2,1,2],
    [5,5,5]
], dtype='double')

b3a = np.array([10,-10,0], dtype='double')[:,None] # force column vector

A3b = np.array([
    [8,14,0],
    [2,2,-6],
    [1,2,1]
], dtype='double')

b3b = np.array([6,5,1], dtype='double')[:,None] # force column vector

A3c = np.array([
    [4,7,0],
    [2,2,-6],
    [1,2,1]
], dtype='double')

b3c = np.array([18,-12,8], dtype='double')[:,None] # force column vector

[unique_soln(x) for x in [A3a, A3b, A3c]];


det(A)!=0 (5.0)-> A is non-singular, a unique solution exists.
det(A)==0 (up to 4 dec) -> A is singular, no unique solution exists.
det(A)==0 (up to 4 dec) -> A is singular, no unique solution exists.


In [35]:
P,L,D,U = LDU(A3b)

Pb = np.matmul(P,b3b)

[print("{}\n".format(x)) for x in [Pb,L,D,U]];

[[6.]
 [5.]
 [1.]]

[[ 1.          0.          0.        ]
 [ 0.25        1.          0.        ]
 [ 0.125      -0.16666667  1.        ]]

[[ 8.   0.   0. ]
 [ 0.  -1.5  0. ]
 [ 0.   0.   0. ]]

[[ 1.    1.75  0.  ]
 [-0.    1.    4.  ]
 [  nan   nan   nan]]



/home/aefrank/applications/anaconda3/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:83: RuntimeWarning: invalid value encountered in true_divide


In [23]:
np.linalg.inv(D)

array([[ 0.2,  0. ,  0. ],
       [-0. , -1. , -0. ],
       [ 0. ,  0. ,  1. ]])

In [36]:
np.linalg.solve(A3b,b3b)

array([[ 1.05083991e+17],
       [-6.00479950e+16],
       [ 1.50119988e+16]])

In [59]:
U,S,V = [np.round(x,4) for x in np.linalg.svd(A3b)]

[print("{}\n".format(x)) for x in [U, S, V]];

[[-0.9736  0.1606 -0.1622]
 [-0.1868 -0.9689  0.1622]
 [-0.1311  0.1882  0.9733]]

[16.5315  6.0587  0.    ]

[[-0.5017 -0.863   0.0599]
 [-0.0767  0.1133  0.9906]
 [-0.8616  0.4924 -0.1231]]



In [62]:
# Computing A+ from ill-conditioned A
# based on https://www.cse.unr.edu/~bebis/MathMethods/SVD/lecture.pdf
t = 0.01
D0_inv = np.diag([1/x if x > t else 0 for x in S])
print(D0_inv)
xbar = np.matmul(V,np.matmul(D0_inv,np.matmul(np.transpose(U),b3b)))
xbar

[[0.06049058 0.         0.        ]
 [0.         0.16505191 0.        ]
 [0.         0.         0.        ]]


array([[ 0.73559282],
       [-0.03701038],
       [ 0.05985662]])

In [63]:
np.matmul(A3b,xbar)

array([[5.36659721],
       [1.03802519],
       [0.72142867]])

In [65]:
b3b

array([[6.],
       [5.],
       [1.]])